In [47]:
import pandas as pd
import json

In [48]:
data_column=[ 'Postal Code', 'Borough', 'Neighborhood']

In [6]:
#data_wiki_df.head()

,Postal Code,Borough,Neighborhood


In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(URL)
print(response)
soup = BeautifulSoup(response.text,'html.parser')

right_table = soup.find('table',{'class':'wikitable sortable'})

#number of columns
for row in right_table.findAll("tr"):
    cells = row.findAll('td')

len(cells)

#number of rows

rows= right_table.findAll("tr")
len(rows)
#print(table)
#rows = table.find_all('tr')
header = [th.text.rstrip() for th in rows[0].find_all('th')]
print(header)
print('------------')
print(len(header))




<Response [200]>
['Postal Code', 'Borough', 'Neighbourhood']
------------
3


In [30]:
c1=[]
c2=[]
c3=[]
for row in right_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        c1.append(cells[0].find(text=True))
        c2.append(cells[1].find(text=True))  # fetch the text of the url in td tag. 
        c3.append(cells[2].find(text=True))

In [31]:
d = dict([(x,0) for x in header])
d

{'Postal Code': 0, 'Borough': 0, 'Neighbourhood': 0}

In [32]:
d['Postal Code']=c1
d['Borough']=c2
d['Neighbourhood']=c3

In [43]:
#covert dict to dataframe
df = pd.DataFrame(d)

df.drop([0],axis=0,inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


In [45]:

df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


In [46]:
df.shape

(179, 3)